In [28]:
# HEAVILY DRAWN FROM: 
# https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
import pandas as pd
import tensorflow as tf
import os
BATCH_SIZE = 2

In [29]:
training_path = os.path.join("./", "final_results/", "trainment.csv")
df = pd.read_csv(training_path)
df.tail()

,ticker,polarity_title,subjectivity_title,polarity_description,subjectivity_description,Open,Close,Open-Close,target
1592,YELP,0.000,0.100000,0.182567,0.392550,29.440001,30.33,-0.889999,0
1593,YELP,0.000,0.000000,0.180139,0.416358,29.440001,30.33,-0.889999,0
1594,YELP,0.075,0.366667,0.075000,0.366667,29.440001,30.33,-0.889999,0
1595,YELP,0.000,0.000000,0.132656,0.404388,29.440001,30.33,-0.889999,0
1596,YELP,0.000,0.000000,0.099260,0.424278,29.440001,30.33,-0.889999,0


In [30]:
target = df.pop('target')

In [31]:
numeric_feature_names = [
    'polarity_title', 'subjectivity_title',
    'polarity_description', 'subjectivity_description', 
    'Open', 'Close', 'Open-Close' 
]
numeric_features = df[numeric_feature_names]
numeric_features.head()

,polarity_title,subjectivity_title,polarity_description,subjectivity_description,Open,Close,Open-Close
0,0.0,0.0,0.254308,0.624943,105.040001,105.830002,-0.790001
1,-0.5,0.7,0.131250,0.479861,105.040001,105.830002,-0.790001
2,0.0,0.0,0.151176,0.502663,105.040001,105.830002,-0.790001
3,0.0,0.0,0.098940,0.376537,105.040001,105.830002,-0.790001
4,0.5,0.5,0.100000,0.600000,105.040001,105.830002,-0.790001


In [32]:
tf.convert_to_tensor(numeric_features)

<tf.Tensor: shape=(1597, 7), dtype=float64, numpy=
array([[ 0.00000000e+00,  0.00000000e+00,  2.54308390e-01, ...,
         1.05040001e+02,  1.05830002e+02, -7.90000916e-01],
       [-5.00000000e-01,  7.00000000e-01,  1.31250000e-01, ...,
         1.05040001e+02,  1.05830002e+02, -7.90000916e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.51176236e-01, ...,
         1.05040001e+02,  1.05830002e+02, -7.90000916e-01],
       ...,
       [ 7.50000000e-02,  3.66666667e-01,  7.50000000e-02, ...,
         2.94400005e+01,  3.03299999e+01, -8.89999390e-01],
       [ 0.00000000e+00,  0.00000000e+00,  1.32656396e-01, ...,
         2.94400005e+01,  3.03299999e+01, -8.89999390e-01],
       [ 0.00000000e+00,  0.00000000e+00,  9.92604618e-02, ...,
         2.94400005e+01,  3.03299999e+01, -8.89999390e-01]])>

In [33]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(numeric_features)

In [34]:
def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

model = get_basic_model()
model.fit(numeric_features, target, epochs=15, batch_size=BATCH_SIZE)

Epoch 1/15
799/799 [==============================] - 13s 15ms/step - loss: 0.4076 - accuracy: 0.8096
Epoch 2/15
799/799 [==============================] - 12s 14ms/step - loss: 0.1506 - accuracy: 0.9468
Epoch 3/15
799/799 [==============================] - 11s 14ms/step - loss: 0.0909 - accuracy: 0.9668
Epoch 4/15
799/799 [==============================] - 11s 14ms/step - loss: 0.0664 - accuracy: 0.9737
Epoch 5/15
799/799 [==============================] - 10s 12ms/step - loss: 0.0541 - accuracy: 0.9812
Epoch 6/15
799/799 [==============================] - 11s 13ms/step - loss: 0.0452 - accuracy: 0.9875
Epoch 7/15
799/799 [==============================] - 9s 11ms/step - loss: 0.0403 - accuracy: 0.9875
Epoch 8/15
799/799 [==============================] - 9s 11ms/step - loss: 0.0344 - accuracy: 0.9900
Epoch 9/15
799/799 [==============================] - 11s 14ms/step - loss: 0.0310 - accuracy: 0.9912
Epoch 10/15
799/799 [==============================] - 8s 10ms/step - loss: 0.0285 -

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 7)                15        
 ion)                                                            
                                                                 
 dense_3 (Dense)             (None, 10)                80        
                                                                 
 dense_4 (Dense)             (None, 10)                110       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 216
Trainable params: 201
Non-trainable params: 15
_________________________________________________________________


In [36]:
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


INFO:tensorflow:Assets written to: saved_model/my_model/assets
